In [22]:
from simpletransformers.classification import ClassificationModel
import pandas as pd
import sklearn
import logging

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=4

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=4


In [23]:
train_args = {
    "output_dir": "outputs_rnd/",
    "cache_dir": "cache/",
    "best_model_dir": "outputs_rnd/best_model/",

    "fp16": False,
    "fp16_opt_level": "O1",
    "max_seq_length": 512,
    "train_batch_size": 4,
    "eval_batch_size": 8,
    "gradient_accumulation_steps": 1,
    "num_train_epochs": 4,
    "weight_decay": 0,
    "learning_rate": 5e-6,
    "adam_epsilon": 1e-9,
    "warmup_ratio": 0.06,
    "warmup_steps": 0,
    "max_grad_norm": 1.0,
    "do_lower_case": False,

    "logging_steps": 50,
    "evaluate_during_training": True,
    "evaluate_during_training_steps": 0,
    "evaluate_during_training_verbose": True,
    "use_cached_eval_features": False,
    "save_eval_checkpoints": False,
    "save_steps": 0,
    "no_cache": True,
    "save_model_every_epoch": True,
    "tensorboard_dir": None,

    "overwrite_output_dir": True,
    "reprocess_input_data": True,

    "n_gpu": 1,
    "silent": False,
    "use_multiprocessing": True,

    "wandb_project": None,
    "wandb_kwargs": {},

    "use_early_stopping": False,
    "early_stopping_patience": 4,
    "early_stopping_delta": 0,
    "early_stopping_metric": "f1",
    "early_stopping_metric_minimize": False,

    "manual_seed": 9721,
    "encoding": None,
    "config": {},
}

In [24]:
import pickle
with open("data/distinct_sets/within-v2.pkl", "rb") as f:
    distinct_train_df = pickle.load(f)
    distinct_dev_df = pickle.load(f)
    overlap_train_df = pickle.load(f)
    overlap_dev_df = pickle.load(f)
    rnd_train_df = pickle.load(f)
    rnd_dev_df = pickle.load(f)

In [25]:
def precision_macro(y_true, y_pred):
    return sklearn.metrics.precision_score(y_true, y_pred, average='macro')
def recall_macro(y_true, y_pred):
    return sklearn.metrics.recall_score(y_true, y_pred, average='macro')
def f1_macro(y_true, y_pred):
    return sklearn.metrics.f1_score(y_true, y_pred, average='macro')

In [34]:
# train
train_df = distinct_train_df[['argument1', 'argument2', 'is_same_side']].copy()
train_df["is_same_side"] = train_df["is_same_side"].astype(int)
train_df = train_df.rename(columns={"argument1": "text_a", "argument2": "text_b", "is_same_side" : 'labels'})
train_df.head(2)

,text_a,text_b,labels
id,,,
12972,"! round 1 is for acceptance only, no arguments...",i thank con for their reply.freedom not restri...,1
12973,"! round 1 is for acceptance only, no arguments...",i thank sengejuri for their opening argument.f...,1


In [35]:
# dev
dev_df = distinct_dev_df[['argument1', 'argument2', 'is_same_side']].copy()
dev_df["is_same_side"] = dev_df["is_same_side"].astype(int)
dev_df = dev_df.rename(columns={"argument1": "text_a", "argument2": "text_b", "is_same_side" : 'labels'})
dev_df.head(2)

,text_a,text_b,labels
id,,,
3921,"!!! round 1 is for acceptance only, no argumen...",i realized my opponent has made his case entir...,0
3897,"!!! round 1 is for acceptance only, no argumen...",premise 1) freedom is our defaultpro never arg...,1


In [36]:
# Create a ClassificationModel
model = ClassificationModel('bert', 'bert-base-uncased', num_labels=2, use_cuda=True, cuda_device=0, args=train_args)

In [ ]:
# Train the model
model.train_model(train_df, eval_df=dev_df, precision=precision_macro, recall=recall_macro, f1=f1_macro)

INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.


Running loss: 1.388087


INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.
INFO:simpletransformers.classification.classification_model:{'mcc': 0.3809563874745878, 'tp': 990, 'tn': 2112, 'fp': 515, 'fn': 761, 'precision': 0.6964636962532365, 'recall': 0.6846750467458151, 'f1': 0.6880540540540541, 'eval_loss': 0.8110922449778248}


Running loss: 0.006702


INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.
INFO:simpletransformers.classification.classification_model:{'mcc': 0.4205293340323079, 'tp': 1077, 'tn': 2098, 'fp': 529, 'fn': 674, 'precision': 0.7137322342801795, 'recall': 0.7068533571658546, 'f1': 0.7094126423636854, 'eval_loss': 1.0400738383432593}


Running loss: 1.405666

In [ ]:
model = ClassificationModel('bert', train_args['best_model_dir'], num_labels=2, use_cuda=True, cuda_device=0, args=train_args)

In [ ]:
# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(
    dev_df, acc=sklearn.metrics.accuracy_score, precision=precision_macro, recall=recall_macro, f1=f1_macro
)

In [ ]:
predict_df = [[row['text_a'], row['text_b']] for index, row in dev_df[1:10].iterrows()]

In [ ]:
predictions, raw_outputs = model.predict(predict_df)
print(predictions)
print(raw_outputs)